For simplicity, and because we wanted to collect data from a large sample regardless of the audit's completion, we drew one round with a high stopping probability.
However, on average fewer ballots need to be sampled when samples are drawn in multiple smaller rounds.
In this notebook we explore how audits would have transpired for the sample we drew, if it had been drawn in smaller rounds.

In [14]:
# Imports
from r2b2.contest import ContestType, Contest
from r2b2.minerva2 import Minerva2
from r2b2.minerva import Minerva
from r2b2.eor_bravo import EOR_BRAVO
from r2b2.so_bravo import SO_BRAVO
import numpy as np
import pandas as pd

In [15]:
# First, let's set all the same audit parameters and contest information
contest_name = "\nSchool Construction and Renovation Projects"
tally = {'Approve' : 2391, 'Reject' : 1414}
risk_limit = .1
reported_winner = max(tally, key=tally.get) 
winner_votes = tally[reported_winner]
total_relevant = sum(tally.values())
loser_votes = total_relevant - winner_votes
margin = (winner_votes / total_relevant) - (loser_votes / total_relevant)
contest_reported = Contest(total_relevant, 
                            tally, 
                            num_winners=1, 
                            reported_winners=[reported_winner],
                            contest_type=ContestType.PLURALITY)

In [16]:
# let's see if we can get the selection order from the 'interim report'
# we will test by using the interim report from Pennsylvania
filename = 'interim-report.csv'
df = pd.read_csv(filename)
df.head()
#sort = tst.sort(["Mean"], ascending = False)
df = df.sort_values(by="Ticket Numbers", ascending=True)
# save dataframe as csv to send to Filip and Poorvi for cross-checking risk values
df.to_csv('Selection-Ordered-Sample.csv', index=False) 

approve_so = []
reject_so = []
for item in df["Audit Result"]: #change to name of column
    if item == 'Approve':
        approve_so.append(1)
        reject_so.append(0)
    elif item == 'Reject':
        reject_so.append(1)
        approve_so.append(0)
sample = {
    'Approve': sum(approve_so),
    'Reject': sum(reject_so),
    'Approve_so': approve_so,
    'Reject_so': reject_so,
}
print(sample['Approve'])
print(sample['Reject'])

MAXIMUM_POSSIBLE_SAMPLE = sample['Approve'] + sample['Reject']

81
59


In [17]:
# Here is a function to run any hypothetical round schedule and print the results
def hypothetical_round_schedule(sample, round_schedule):
    # Divide the sample according to this round schedule
    samples = []
    for round_size in round_schedule:
        approve_so = sample['Approve_so'][0:round_size]
        approve = sum(approve_so)
        reject_so = sample['Reject_so'][0:round_size]
        reject = sum(reject_so)
        samples.append({'Approve_so':approve_so, 'Approve':approve, 'Reject_so':reject_so, 'Reject':reject})
    # Now create new audit objects to run this hypothetical audit with each of the BRAVOs and Minervas
    minerva2 = ('Minerva 2.0', Minerva2(risk_limit, 1.0, contest_reported))
    minerva = ('Minerva', Minerva(risk_limit, 1.0, contest_reported))
    so_bravo = ('Selection-Ordered BRAVO', SO_BRAVO(risk_limit, 1.0, contest_reported))
    eor_bravo = ('End-of-Round BRAVO', EOR_BRAVO(risk_limit, 1.0, contest_reported))
    audits = [minerva2, minerva, so_bravo, eor_bravo]
    # Run the hypothetical audits, printing the results
    for audit_name, audit in audits:
        print('{}:'.format(audit_name))
        for i, (round_size, round_sample) in enumerate(zip(round_schedule, samples)):
            audit.execute_round(round_size, round_sample)
            print('Round {}: {} total, {} winner: risk {} -- stopped: {}'.format(i+1, round_size, round_sample['Approve'], round(audit.pvalue_schedule[-1], 4), audit.stopped))
            if audit.stopped:
                break

In [18]:
# Function for getting a round schedule corresponding to a divisor
def round_schedule(divisor):
    marginal_round_schedule = [len(sample['Approve_so']) // divisor] * (divisor - 1)
    marginal_round_schedule.append(len(sample['Approve_so']) - (len(sample['Approve_so']) // divisor) * (divisor - 1))
    cumulative_round_schedule = [sum(marginal_round_schedule[0:i+1]) for i in range(len(marginal_round_schedule))]
    return cumulative_round_schedule

In [19]:
# Let's try the function above
hypothetical_round_schedule(sample, round_schedule(2))

Minerva 2.0:
Round 1: 70 total, 44 winner: risk 0.0375 -- stopped: True
Minerva:
Round 1: 70 total, 44 winner: risk 0.0375 -- stopped: True
Selection-Ordered BRAVO:
Round 1: 70 total, 44 winner: risk 0.0899 -- stopped: True
End-of-Round BRAVO:
Round 1: 70 total, 44 winner: risk 0.0963 -- stopped: True


In [20]:
# While Minerva 1.0 requires predetermined round sizes, Minerva 2.0 could run picking round sizes that achieve some desired stopping probability
# Here's a similar hypothetical round schedule function but now round sizes are determined by some stopping probability
def hypothetical_by_sprob(sample, sprob):
    # Now create new audit objects to run this hypothetical audit with each of the BRAVOs and Minervas
    minerva2 = ('Minerva 2.0', Minerva2(risk_limit, 1.0, contest_reported))
    minerva = ('Minerva', Minerva(risk_limit, 1.0, contest_reported))
    so_bravo = ('Selection-Ordered BRAVO', SO_BRAVO(risk_limit, 1.0, contest_reported))
    eor_bravo = ('End-of-Round BRAVO', EOR_BRAVO(risk_limit, 1.0, contest_reported))
    audits = [minerva2, minerva, so_bravo, eor_bravo]
    # Run the hypothetical audits, printing the results
    for audit_name, audit in audits:
        print('\n{}:'.format(audit_name))
        round_num = 0
        while True:
            round_num += 1
            # Determine the next round size
            if audit_name == 'Minerva' and round_num > 1:
                round_size = int(round_size + round_size * 1.5)
            else:
                round_size = audit.next_sample_size(sprob)
            if round_size > MAXIMUM_POSSIBLE_SAMPLE:
                print('Next round size would be {} which exceeds the sample we drew of {}'.format(round_size, MAXIMUM_POSSIBLE_SAMPLE))
                break

            # Get the sample for this round size
            approve_so = sample['Approve_so'][0:round_size]
            approve = sum(approve_so)
            reject_so = sample['Reject_so'][0:round_size]
            reject = sum(reject_so)
            cur_sample = {'Approve_so':approve_so, 'Approve':approve, 'Reject_so':reject_so, 'Reject':reject}

            # Execute the round
            audit.execute_round(round_size, cur_sample)
            print('Round {}: {} total, {} winner: risk {} -- stopped: {}'.format(round_num, round_size, cur_sample['Approve'], round(audit.pvalue_schedule[-1], 4), audit.stopped))
            if audit.stopped:
                break

In [21]:
# Try out this new function
hypothetical_by_sprob(sample, .5)


Minerva 2.0:
Round 1: 44 total, 28 winner: risk 0.0918 -- stopped: True

Minerva:
Round 1: 44 total, 28 winner: risk 0.0918 -- stopped: True

Selection-Ordered BRAVO:
Round 1: 52 total, 30 winner: risk 0.7205 -- stopped: False
Round 2: 96 total, 59 winner: risk 0.0539 -- stopped: True

End-of-Round BRAVO:
Round 1: 70 total, 44 winner: risk 0.0963 -- stopped: True
